## Deep Learning Steps - PoseNet
Both scripts aim to detect exercise segments based on PoseNet data. They share several core steps:

1. **Data Loading & Preprocessing**  
   - Read full-pose CSV files and corresponding trimmed CSVs.  
   - Smooth sequences via a moving-average window and compute deltas.  
   - Derive labels indicating exercise frames vs. non-exercise frames.

2. **Train/Test Split**  
   - Group-based splitting ensures entire videos fall into either train or test.

3. **Feature Scaling**  
   - A `StandardScaler` is fit on training data only, then applied to both sets.

4. **Model Definition & Training**  
   - The first script trains an MLP classification model (using scikeras + GridSearchCV).  
   - The second script handles two modes:  
     - “frame” mode with a weighted-BCE loss per frame.  
     - “boundary” mode using MSE on the normalized start/end positions.

5. **Evaluation**  
   - Compute frame-level metrics like precision, recall, F1, and boundary errors.  
   - In the second script, boundary-mode evaluation checks how close predicted start/end match the ground-truth.

###  Parameter Variations
1. **First Script (postnetcutting.py)**  
   - Tries a grid of MLP hyperparameters (hidden_units=256, hidden_layers=12, dropout_rate=0.30).  
   - Uses up to 50 epochs and a batch size of 64.

2. **Second Script (postnetcutting2.py)**  
   - Selects number of layers (default 12) and hidden units (128).  
   - Adjusts mode (“frame” / “boundary”) for loss function choice.  
   - Configures distinct hyperparameters (dropout=0, LR=1e-4, batch size=256, epochs=50).

### Results
After training, the aggregate boundary error was:

──────── Aggregate boundary error ────────

Δstart mean +5.33 ± 25.88 median |Δ| 12.0 frames

Δend mean -3.28 ± 21.89 median |Δ| 8.5 frames

This indicates that on average, the model predicts start frames 5.33 frames later and end frames 3.28 frames earlier than the ground truth. The high standard deviations (25.88 and 21.89 frames) suggest considerable variability in predictions. The median absolute errors (12.0 for start, 8.5 for end) provide a more robust measure of typical error, less affected by outliers.


## Deep Learning Steps - Kinect

Based on the analysis of kinectcutting.py, the following steps were taken:

1. Data Loading and Preprocessing

* Loads matched CSV files from uncut and preprocessed directories.
* Applies data augmentation: mirroring and rotation (±15 degrees).
* Creates sliding windows of 11 frames for input features.

2. Data Splitting

* Uses GroupShuffleSplit to ensure entire sequences remain in either train or test set.
* 80% train, 20% test split.

3. Feature Scaling

* Applies StandardScaler to normalize input features.

4. Model Architecture

* Implements a sequential model with multiple dense layers.
* Uses ReLU activation for hidden layers and sigmoid for output.
* Compiles with binary cross-entropy loss and Adam optimizer.

5. Hyperparameter Tuning

* Utilizes GridSearchCV for hyperparameter optimization.
* Searches over number of layers, units, learning rate, batch size, and epochs.

6. Model Training

* Trains the model using the best hyperparameters from GridSearchCV.
* Implements early stopping to prevent overfitting.

7. Evaluation

* Evaluates on test set for loss, accuracy, precision, and recall.
* Performs detailed boundary error analysis per video.

### Parameter Variations
The script explores the following hyperparameter space:

* Number of layers: [12]
* Number of units per layer: [128]
* Learning rate: [1e-4]
* Batch size: [128]
* Number of epochs: [50]
* Early stopping is implemented with a patience of 5 epochs, monitoring validation loss.

### Result

───────── Aggregate boundary error ─────────

Δstart  mean  +2.83 ± 19.14   median |Δ| 5.0 frames

Δend    mean  +1.22 ± 16.01   median |Δ| 3.0 frames

### These results indicate:

Start frames are predicted, on average, 2.83 frames later than actual.
End frames are predicted, on average, 1.22 frames later than actual.
The median absolute errors (5.0 for start, 3.0 for end) suggest relatively good accuracy in boundary detection.
Standard deviations (19.14 for start, 16.01 for end) indicate some variability in predictions.
Overall, the model shows promising performance in detecting exercise segment boundaries, with slightly better accuracy for end frames compared to start frames.




## Software development